In [13]:
import pandas as pd
import numpy as np

campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')

# Goals 
1. the core task we're interested in is identifying those customers most likely to subscribe to a term deposit

 

2. actionable patterns in data

 

3. does frequent contact have an adverse effect

 

4. seperate models for historic economic indicators

 

5. the model should be saved to a file

 

6. run the model on a different set of data and make target predictions

In [14]:
campaign.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [15]:
campaign.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [16]:
yes = campaign[ campaign['y'] == 'yes']
yes['day_of_week'].value_counts()

day_of_week
thu    958
wed    865
tue    857
fri    767
mon    761
Name: count, dtype: int64

In [17]:
campaign.replace({'no':0,'yes':1},inplace=True)

In [18]:
from typing_extensions import dataclass_transform
'''import altair as alt

day = campaign.groupby(["day_of_week","y"])["day_of_week"].count()
day.head(15)

alt.Chart(day).mark_bar().transform_calculate().encode(
    alt.X('y:N', title='Survived', axis=alt.Axis(labelFontSize=14, labelAngle=0)),
    alt.Y('', title='Total Survivors'),
    column=alt.Column('day_of_week:N', title=None)
).properties(
    title='How Did Passenger Gender Affect Survival',
    width=100
)'''

'import altair as alt\n\nday = campaign.groupby(["day_of_week","y"])["day_of_week"].count()\nday.head(15)\n\nalt.Chart(day).mark_bar().transform_calculate().encode(\n    alt.X(\'y:N\', title=\'Survived\', axis=alt.Axis(labelFontSize=14, labelAngle=0)),\n    alt.Y(\'\', title=\'Total Survivors\'),\n    column=alt.Column(\'day_of_week:N\', title=None)\n).properties(\n    title=\'How Did Passenger Gender Affect Survival\',\n    width=100\n)'

In [19]:
check = campaign[["default", "housing", "loan"]]
check.apply(pd.Series.value_counts)
campaign = campaign.replace("unknown", np.nan)
#(campaign==np.nan).sum()
campaign.isna().sum()

age                  0
job                294
marital             69
education         1535
default           7725
housing            894
loan               894
contact              0
month                0
day_of_week          0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64

In [20]:
# both default and poutcome have so little useable data in comparison to missing data or such a poor balance of values that I don't think they are worth using in training.
campaign = campaign.drop(["default", "poutcome"], axis=1)
campaign["housing"] = campaign["housing"].astype(float)
campaign["loan"] = campaign["loan"].astype(float)
campaign.dtypes

age                 int64
job                object
marital            object
education          object
housing           float64
loan              float64
contact            object
month              object
day_of_week        object
campaign            int64
pdays               int64
previous            int64
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                   int64
dtype: object

In [21]:

campaign["education"]=campaign["education"].replace(["illiterate","basic.4y","basic.6y","basic.9y","high.school","professional.course","university.degree"],[1,2,3,4,5,6,7])
campaign["education"].value_counts()
campaign = pd.get_dummies(campaign,prefix=["job"],columns=["job"])
campaign = pd.get_dummies(campaign,prefix=["marital"],columns=["marital"])
campaign = pd.get_dummies(campaign,prefix=["contact"],columns=["contact"])
#campaign["job"] = campaign["job"].replace(["unemployed","student","services","housemaid","blue-collar","technician","admin.","management","self-employed","entrepreneur","retired"],[0,1,2,3,4,5,6,7,8,9,10])
#campaign["job"].value_counts()


In [22]:
campaign['day_of_week'] = campaign['day_of_week'].replace(["mon","tue","wed","thu","fri"],[1,2,3,4,5]) 
campaign['month'] = campaign['month'].replace(["jan","feb","mar","apr", "may","jun", "jul", "aug", "sep", "oct", "nov", "dec"],[1,2,3,4,5,6,7,8,9,10,11,12]) 
print(campaign['day_of_week'].value_counts())
print(campaign['month'].value_counts())


day_of_week
4    7742
1    7657
3    7347
2    7287
5    7036
Name: count, dtype: int64
month
5     12370
7      6445
8      5555
6      4817
11     3698
4      2369
10      653
9       508
3       496
12      158
Name: count, dtype: int64


In [23]:
campaign.dtypes

age                    int64
education            float64
housing              float64
loan                 float64
month                  int64
day_of_week            int64
campaign               int64
pdays                  int64
previous               int64
emp.var.rate         float64
cons.price.idx       float64
cons.conf.idx        float64
euribor3m            float64
nr.employed          float64
y                      int64
job_admin.              bool
job_blue-collar         bool
job_entrepreneur        bool
job_housemaid           bool
job_management          bool
job_retired             bool
job_self-employed       bool
job_services            bool
job_student             bool
job_technician          bool
job_unemployed          bool
marital_divorced        bool
marital_married         bool
marital_single          bool
contact_cellular        bool
contact_telephone       bool
dtype: object

In [24]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mean.fit(campaign[['loan','housing']])
campaign[['loan','housing']] = imp_mean.transform(campaign[['loan','housing']])
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(campaign[["education"]])
campaign[["education"]] = imp_mode.transform(campaign[["education"]])
campaign.isna().sum()

age                  0
education            0
housing              0
loan                 0
month                0
day_of_week          0
campaign             0
pdays                0
previous             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
job_admin.           0
job_blue-collar      0
job_entrepreneur     0
job_housemaid        0
job_management       0
job_retired          0
job_self-employed    0
job_services         0
job_student          0
job_technician       0
job_unemployed       0
marital_divorced     0
marital_married      0
marital_single       0
contact_cellular     0
contact_telephone    0
dtype: int64

In [26]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Encode our features and target as needed
features = ["age",
"education",
"housing",
"loan",
"month",
"day_of_week",
"campaign",
"pdays",
"emp.var.rate",
"cons.price.idx",
"cons.conf.idx",
"euribor3m",
"nr.employed",
"job_admin.",
"job_blue-collar",
"job_entrepreneur",
"job_housemaid",
"job_management",
"job_retired",
"job_self-employed",
"job_services",
"job_student",
"job_technician",
"job_unemployed",
"marital_divorced",
"marital_married",
"marital_single",
"contact_cellular",
"contact_telephone"]
X = pd.get_dummies(campaign[features], drop_first=True)
y = campaign['y']

# Split our data into training and test data, with 30% reserved for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Build the decision tree
clf = DecisionTreeClassifier()

# Train it
clf.fit(X_train, y_train)

# Test it 
clf.score(X_test, y_test)

0.8406618109882205

In [37]:
import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)

In [38]:
campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank_holdout_test.csv')
clf2 = pickle.loads(s)

campaign.replace({'no':0,'yes':1},inplace=True)

campaign = campaign.replace("unknown", np.nan)

campaign = campaign.drop(["default", "poutcome"], axis=1)
campaign["housing"] = campaign["housing"].astype(float)
campaign["loan"] = campaign["loan"].astype(float)

campaign["education"]=campaign["education"].replace(["illiterate","basic.4y","basic.6y","basic.9y","high.school","professional.course","university.degree"],[1,2,3,4,5,6,7])
campaign["education"].value_counts()
campaign = pd.get_dummies(campaign,prefix=["job"],columns=["job"])
campaign = pd.get_dummies(campaign,prefix=["marital"],columns=["marital"])
campaign = pd.get_dummies(campaign,prefix=["contact"],columns=["contact"])

campaign['day_of_week'] = campaign['day_of_week'].replace(["mon","tue","wed","thu","fri"],[1,2,3,4,5]) 
campaign['month'] = campaign['month'].replace(["jan","feb","mar","apr", "may","jun", "jul", "aug", "sep", "oct", "nov", "dec"],[1,2,3,4,5,6,7,8,9,10,11,12]) 

imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mean.fit(campaign[['loan','housing']])
campaign[['loan','housing']] = imp_mean.transform(campaign[['loan','housing']])
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(campaign[["education"]])
campaign[["education"]] = imp_mode.transform(campaign[["education"]])

clf2.predict(campaign)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- previous
